In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Exploracion de datos

## Application train

Importamos el dataset de entrenamiento, y revisamos sus estadisticas principales rapidamente.

In [ ]:
data_train = pd.read_csv('Data/application_train.csv')
print(data_train.shape)
data_train.head()

Al revisar las estadisticas que muestra la sentencia *data_train.describe()* observamos algo extraño en algunas columnas, como la columna *Days_Birth* que tiene valores negativos, y esto se debe a que las edades estan calculadas en dias y de manera relativa a la aplicación del prestamo.

In [ ]:
data_train.describe()

Para arreglar esta irregularidad en los datos, sacamos el valor absoluto de los dias que nos presenta el dataset, y para una vision mas comun de la edad, lo convertimos en años dividiendo por 365.

In [ ]:
data_train['DAYS_BIRTH'] = abs(data_train['DAYS_BIRTH'])/365

In [ ]:
#revisamos con un boxplot que todo este mas o menos normal y no existan outliers exagerados
plt.boxplot(data_train['DAYS_BIRTH'])
plt.title('Boxplot de las edades')
plt.ylabel('Edad en años');

Todo parece estar bien con las edades, ahora haremos lo mismo con la columna de *DAYS_EMPLOYED* que posee la misma irregularidad

In [ ]:
data_train['DAYS_EMPLOYED'] = abs(data_train['DAYS_EMPLOYED'])/365

In [ ]:
plt.boxplot(data_train['DAYS_EMPLOYED'])
plt.title('Boxplot de los años de trabajo')
plt.ylabel('Años que lleva en el empleo actual.');

Y se puede observar que existen outliers, y unos muy exagerados he ilogicos, tanto asi que hay algunos que han estado 1000 años en su ultimo empleo.
- Estos Outliers se eliminaran inmediatamente, y quedaran como valores *NaN* porque es ilogico e imposible que una persona haya trabajado o vivido 1000 años.
- Otra cosa importante, es que estos valores extraños, son todos iguales y equivalen a 1000.6657534246575 años de trabajo. **ENTONCES** cuando rellenemos estos valores *NaN* que crearemos, todos van a ser llenados con el mismo valor.

In [ ]:
weird_data = data_train[data_train['DAYS_EMPLOYED'] >= 1000]
('Los datos raros corresponden a un %0.2f%% de todos los datos.' % (100*weird_data['DAYS_EMPLOYED'].shape[0]/data_train.shape[0]))

In [ ]:
data_train['DAYS_EMPLOYED'].replace(1000.6657534246575, np.nan, inplace=True)

In [ ]:
#Y graficamos de nuevo para ver el nuevo boxplot, llenando los Valores NaN con el promedio de la columna.
Days_Job = data_train['DAYS_EMPLOYED'].copy()
Days_Job.fillna(Days_Job.mean(), inplace=True)
print('Se siguen observando Outliers, pero mas logicos, ya que hay personas de mas de 60 años, es posible que lleven 40 años en el mismo trabajo')
plt.boxplot(Days_Job)
plt.ylabel('Años que lleva en el empleo actual');

- Revisamos las otras columnas que tienen el mismo formato de dias relativos a la aplicacion del credito, y las transformamos a valores positivos.
- Los Outliers presentados en estas columnas se pueden pasar por alto, ya que son posibles.

In [ ]:
data_train['DAYS_REGISTRATION'] = abs(data_train['DAYS_REGISTRATION'])

In [ ]:
plt.boxplot(data_train['DAYS_REGISTRATION']);

In [ ]:
data_train['DAYS_ID_PUBLISH'] = abs(data_train['DAYS_ID_PUBLISH'])

In [ ]:
plt.boxplot(data_train['DAYS_ID_PUBLISH']);

Graficando se puede ver que nuestro dataset esta muy desbalanceado.

In [ ]:
plt.figure(figsize=(5,3))
plt.hist(data_train['TARGET'].astype(int))
plt.xticks([0,1]);

Explicitamente se puede ver la proporcion de cada posible valor de nuestro TARGET

In [ ]:
print('target 1 :', sum(data_train.TARGET==1), 'datos')
print('target 0 :', sum(data_train.TARGET==0), 'datos')
print('------------------')
print('%target 1 :', np.mean(data_train.TARGET==1))
print('%target 0 :', np.mean(data_train.TARGET==0))

Buscando valores NaN

In [ ]:
print('valores NaN', data_train.isnull().sum().sum())
print('porcentaje de valores NaN en el DF:', 100*data_train.isnull().sum().sum()/(data_train.shape[0]*data_train.shape[1]))

Los valores NaN se trataran justo antes de empezar a realizar las predicciones, esto con el fin de probar varias maneras de llenar los valores NaN y escoger la que mejor rendimiento presente.

__________


Buscando caracteristicas categoricas... obtenemos 16!

In [ ]:
data_train.dtypes.value_counts()

revisando cuantos valores unicos poseen las caracteristicas categoricas

In [ ]:
obj_columns = data_train.dtypes[data_train.dtypes.values == 'object'].index.tolist()

In [ ]:
print(data_train[obj_columns].apply(pd.Series.nunique, axis = 0))
print('cantidad de elementos unicos en las variables categoricas', data_train[obj_columns].apply(pd.Series.nunique, axis = 0).values.sum())

Tienen muy pocos valores unicos

## Application Test

Dataset de testeo. Se le realiza el mismo analisis que al anterior.

A primera vista se ve que el conjunto de testeo es significativamente mas pequeño que el conjunto de entrenamiento.

In [ ]:
data_test = pd.read_csv('Data/application_test.csv')
print(data_test.shape)
data_test.head()

In [ ]:
data_test.describe()

**Checkeo y Correccion de irregularidades en las columnas del dataset**  
Se realiza el mismo proceso que se uso en el dataset de entrenamiento para corregir las irregularidades encontradas.

In [ ]:
data_test['DAYS_BIRTH'] = abs(data_test['DAYS_BIRTH'])/365

In [ ]:
#Todo parece estar en orden con las edades
plt.boxplot(data_test['DAYS_BIRTH'])
plt.title('Boxplot de las edades.')
plt.ylabel('Edad.');

Para los dias de empleo que tiene en el trabajo actual

In [ ]:
data_test['DAYS_EMPLOYED'] = abs(data_test['DAYS_EMPLOYED'])/365

In [ ]:
#Todo parece estar en orden con las edades
plt.boxplot(data_test['DAYS_EMPLOYED'])
plt.title('Boxplot de los años de trabajo')
plt.ylabel('Años que lleva en el empleo actual.');

In [ ]:
data_test['DAYS_EMPLOYED'].replace(1000.6657534246575, np.nan, inplace=True)

In [ ]:
print('Maximo valor: ', data_test['DAYS_EMPLOYED'].max())
print('Minimo valor', data_test['DAYS_EMPLOYED'].min())

Para las otras dos variables del mismo formato.

In [ ]:
data_test['DAYS_REGISTRATION'] = abs(data_test['DAYS_REGISTRATION'])

In [ ]:
print('Maximo valor: ', data_test['DAYS_REGISTRATION'].max())
print('Minimo valor', data_test['DAYS_REGISTRATION'].min())

In [ ]:
data_test['DAYS_ID_PUBLISH'] = abs(data_test['DAYS_ID_PUBLISH'])

In [ ]:
print('Maximo valor: ', data_test['DAYS_ID_PUBLISH'].max())
print('Minimo valor', data_test['DAYS_ID_PUBLISH'].min())

**Reviso los valores NaN**

Se puede notar que la proporcion de valores NaN no varia mucho en comparación al dataset de entrenamiento.

In [ ]:
print('valores NaN', data_test.isnull().sum().sum())
print('porcentaje de valores NaN en el DF es de %0.2f%%' %(100*data_test.isnull().sum().sum()/(data_test.shape[0]*data_test.shape[1])))

**Buscando las columnas categoricas**  
Al tratarse de la parte de un mismo archivo, se puede ver que el dataset de testeo posee las mismas columnas categoricas que el dataset de entrenamiento

In [ ]:
data_test.dtypes.value_counts()

In [ ]:
obj_columns = data_test.dtypes[data_test.dtypes.values == 'object'].index.tolist()

In [ ]:
print(data_test[obj_columns].apply(pd.Series.nunique, axis = 0))
print('cantidad de elementos unicos en las variables categoricas', data_test[obj_columns].apply(pd.Series.nunique, axis = 0).values.sum())

***Aunque si se nota un leve cambio en la cantidad de valores unicos que poseen las columnas categoricas.***

# Pre-Procesado de datos

### Codificación de las caracteristicas categoricas

- Usando LabelEncoder para las caracteristicas con 2 categorias
- Usando OneHotEncoder (para crear variables Dummy) para las caracteristicas que tengan mas de 2 categorias

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label = LabelEncoder()

for i in obj_columns:
    if len(data_train[i].unique()) <= 2:
        #entreo el objeto encoder
        label.fit(data_train[i])
        #aplico la transformacion en los dos DF principales.
        data_train[i] = label.transform(data_train[i])
        data_test[i] = label.transform(data_test[i])

#Ahora codificar las que tienen mas de 2 categorias.
data_train = pd.get_dummies(data_train)
data_test = pd.get_dummies(data_test)

In [ ]:
print('Nuevas dimensiones del DF:', data_train.shape)
print('Nuevas dimensiones del DF:', data_test.shape)

Por esta razon tenemos que modificar los dataset de entrenamiento y prueba, ya que deben tener la misma cantidad de columnas para que el modelo predictivo que se cree nos sirva.  

- Se penso en usar el metodo Join, pero el metodo JOIN nos retornaria un solo dataset, y nosotros necesitamos tener los dos, entonces...
- Se usara el metodo Align para "alinear" los dos archivos y que queden con un mismo numero de columnas, sin tener que necesariamente unirlos.

In [ ]:
#Lo primero es salvar nuestra columna de TARGET, porque esta columna no esta en el dataset de pruebas
Targets = data_train['TARGET']

In [ ]:
#hacemos uso del metodo align, que nos permite "recortar" los dataframes para que queden con la misma
#cantidad de columnas. (Usamos las columnas que tienen en comun los dos dataframes)
#Usamos axis = 1 para que nos realice el proceso por columnas y no por filas.
data_train, data_test = data_train.align(data_test, join='inner', axis=1)

In [ ]:
#Ahora solo hay que agregar de nuevo la columna de los TARGETS a nuestro nuevo dataset de entrenamiento
data_train['TARGET'] = Targets

In [ ]:
print('Nuevas dimensiones del DF:', data_train.shape)
print('Nuevas dimensiones del DF:', data_test.shape)

## Feature Engineering

### Buscando Correlaciones
Este paso es muy importante ya que podemos encontrar que caracteristicas afectan mas a nuestro *TARGET*

In [ ]:
import seaborn as sns

In [ ]:
corrs = data_train.corr()

Nos interesa ver las correlaciones de las variables contra nuestra variable *TARGET*

In [ ]:
sorted_corrs = corrs['TARGET'].sort_values()

In [ ]:
sorted_corrs.head(10)

In [ ]:
sorted_corrs.tail(10)

In [ ]:
#Tomo las correlaciones mas altas y mas bajas.
#Las 3 mas altas son: REGION_RATING_CLIENT_W_CITY, REGION_RATING_CLIENT, y NAME_INCOME_TYPE_Working
High_corr_data = data_train[['TARGET', 'REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT', 'NAME_INCOME_TYPE_Working']]
High_corrs = High_corr_data.corr()
sns.heatmap(High_corr_data, cmap='inferno', annot=True)